Chapter-4 focuses on RNN, Gated Recurrent Units (GRU), Long Short Term Memory(LSTM)

### RNN

RNN is used in 2 senses:

    . General - Any NN built using a Recurrent Computational Graph
    . Special - Concrete architectute of NN
    
2 major disadvantages of fully connected NN:

    . Fixed input - can only accept an input sequence of fixed length
    . Does not perceive the vector as an ordered sequence
    
RNN has concept of hidden state. Hidden state can be treated as internal memory. The hidden state does not try to remember all past values of the seuqnce but only thier effect. Because of the internal memory RNNs can rember important things about thier input, which allow them to be very accurate in predicting future values.

RNN takes the internal memory(hidden state), combines it with an event, and returns the new updated internal memory(hidden state).So when it makes a decision, it considers the current input and also what is has learned from the inputs it received previously.

    ht = tanh(WitXt + bih + Whhh(t-1) + bhh)
    
RNN cell combines information about the current value of the sequence Xi an the previously hidden state hi-1.

In [2]:
import torch.nn as nn

class RNN(nn.Module):
    def __init__(self
                , hidden_sie
                , in_size = 1
                , out_size = 1):
        super(RNN, self).__init__()
        self.rnn = nn.RNN(input_size = in_size
                         , hidden_size = hidden_size
                         , batch_first = True)
        self.fc = nn.Linear(hidden_size, out_size)
    def forward(self, x, h = None):
        out, _ = self.rnn(x, h)
        last_hidden_state = out[:, -1]
        out = self.fc(last_hidden_state)
        return out, last_hidden_state

In [ ]:
import copy
import random
import sys
import numpy as np
import matplotlib.pyplot as plt
import torch
from sklearn.preprocessing import MinMaxScaler

random.seed(1)
torch.manual_seed(1)

## Parameters

# length of sliding window
features = 240
# length of test dataset
test_ts_len = 300
# size of RNN hidden state
rnn_hidden_size = 24
# optimizer learning rate
learning_rate = 0.02
training_epochs = 500

# Prepare datasets for training

ts = get_aep_timeseries()
scaler = MinMaxScaler()
scaled_ts = scaler.fit_transform(ts)
x_train, x_val, x_test, y_train, y_val, y_test = get_training_datasets(scaled_ts
                                                                       , features
                                                                       , test_ts_len
                                                                      )


# Initializing the model
model = RNN(hidden_size = rnn_hidden_size)
model.train()

#Training
optimizer = torch.optim.Adam(params=model.parameters(), lr=learning_rate)
mse_loss = torch.nn.MSELoss()
best_model = None
min_val_loss = sys.maxsize
training_loss = []
validation_loss = []
for t in range(training_epochs):
    prediction, _ = model(x_train)
    loss = mse_loss(prediction, y_train)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    val_prediction, _ = model(x_val)
    val_loss = mes_loss(val_predicton, y_val)
    training_loss.append(loss.item())
    validation_loss.append(val_loss.item())
    if val_loss.item() < min_val_loss:
        best_model = copy.deepcopy(model)
        min_val_loss = val_loss.item()
    if t%50 == 0:
        print(f'epoch {t}: train - {round(loss.item(),4)},' f'val: - {round(val_loss.item(), 4)}')
        
# evaluation
best_model.eval()
_, h_list = best_model(x_val)
# warm hidden state
h = (h_list[-1, :].unsqueeze(-2))
predicted = []
for test_seq in x_test.tolist():
    x = torch.Tensor(data=[test_seq])
    #passing hidden state through each iteration
    y, h = best_model(x, h.unsqueeze(-2))
    unscaled = scaler.inverse_transform(np.array(y.item()).reshape(-1,1))[0][0]
    predict.append(unscaled)

RNN significant disadvantage:

    . Suffers from vanishing gradient problems due to computational complexity
    . Training too slow
    . Difficult to store long-term information in RNN

### Gated Recurrent Unit

    . Store long term information
    . Way to store long-term information in a hidden state is to restrict hidden state updates on each iteration.
     
$r_t$ is an output of a simple NN on two input vectors: $h_(t-1)$ and $x_t$. The variable $r_t$ is responsible for fogetting unimportant parts in the hidden state of GRU.

    0.1 - Value is close to 0, and highly likely corresponding hidden state value will be forgotten
    0 - Means that the corresponding hidden state value will be forgotten
    0.8 - Highly likely corresponding hidden state value will be passed to a further hidden state
    1 - The corresponding hidden state value will be passed to a futher hidden state

### Summary

    . RNN has a hidden state that can be treated as an internal memory of the input sequence.
    . RNN recalculates the hidden state after processing each new input values recurrently.
    . RNN suffers from vanishing gradient problem
    . RNN updates a hidden state on each iteration. This it has no long-term memory.
    . GRU implements the reset gate, which declines some updates in a hidden memory.
    . LSTM passes 2 vectors through each iteration: hidden state and cell state